In [1]:
import pandas as pd
import numpy as np 
import os
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, make_scorer, roc_auc_score, mean_squared_error,mean_absolute_error
from sklearn.cluster import KMeans
from scipy import stats
from sklearn.model_selection import GridSearchCV
from prophet.diagnostics import cross_validation
import joblib
# import optuna
# import xgboost as xgb
import catboost
import lightgbm as lgb
from skopt import BayesSearchCV
from prophet import Prophet
from prophet.diagnostics import performance_metrics
import xgboost as xgb

In [63]:
def y_changer(data, col):
    data = data.rename(columns = {"y": "평균기온"})
    return data

In [34]:
data = pd.read_csv('./prep_data.csv')
data.ds = pd.to_datetime(data.ds)
data = data.rename(columns = {"평균기온":"y"})
data['days'] = data.ds.dt.day
data['years'] = data.ds.dt.year
summit_form = pd.read_csv('./sample_submission.csv')
regressors = [x for x in data.columns if x != "y" and x != "ds"]

In [35]:
data = y_changer(data, "최저기온")

In [66]:
train, test = data.iloc[:-365, :], data.iloc[-365:, :]

X_train , y_train = train[["years", "월", "days","최고기온", "최저기온"]], train.평균기온
X_test, y_test = test[["years", "월", "days","최고기온", "최저기온"]], test.평균기온

In [67]:
train.columns

Index(['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율',
       '평균기온', '월', 'ds', 'days', 'years'],
      dtype='object')

In [109]:
## 훈련용

df = train_nofeature.copy()

# Prophet 모델 초기화
model = Prophet(daily_seasonality=True, seasonality_prior_scale = 0.01)

# 모델에 데이터 피팅
model.fit(df)


In [110]:
default = 2.469308583208345
forecast = model.predict(test_x)
answer = forecast.yhat
mean_absolute_error(test_y, answer)
mae = mean_absolute_error(test_y, answer)
print(default - mae)

-0.008745691311886628


In [85]:
## 제출용

df = data[["ds","y"]].copy()


# 모델에 데이터 피팅
model.fit(df)

# 미래 날짜 생성
future = model.make_future_dataframe(periods=358)[-358:]  # 예측할 기간 설정 (365일)

# 예측
forecast = model.predict(future)

summit_form['평균기온']  = model.predict(future).yhat

summit_form.to_csv(f"./answer/prophet_mae_{mae}.csv")

In [113]:
summit_form

,일시,평균기온
0,2023-01-01,-2.397883
1,2023-01-02,-2.507636
2,2023-01-03,-2.635723
3,2023-01-04,-2.706753
4,2023-01-05,-2.747967
...,...,...
353,2023-12-20,-0.918986
354,2023-12-21,-1.032896
355,2023-12-22,-1.172703
356,2023-12-23,-1.241481


In [23]:
train

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,y,월,ds,days,years
0,2.2,-5.2,7.4,0.34,68.3,1.7,6.7,7.029,63.10,-1.6,1.0,1960-01-01,1,1960
1,1.2,-5.6,6.8,0.40,87.7,1.3,0.0,2.887,4.19,-1.9,1.0,1960-01-02,2,1960
2,8.7,-2.1,10.8,0.00,81.3,3.0,0.0,5.854,12.87,4.0,1.0,1960-01-03,3,1960
3,10.8,1.2,9.6,0.00,79.7,4.4,2.6,6.279,21.55,7.5,1.0,1960-01-04,4,1960
4,1.3,-8.2,9.5,0.05,44.0,5.1,8.2,13.086,81.51,-4.6,1.0,1960-01-05,5,1960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22641,-3.9,-12.9,9.0,0.00,60.9,1.7,3.8,6.090,39.60,-7.6,12.0,2021-12-27,27,2021
22642,-0.9,-8.5,7.6,0.66,73.8,2.2,1.7,4.320,17.70,-4.1,12.0,2021-12-28,28,2021
22643,5.9,-3.8,9.7,0.20,72.9,2.6,1.8,4.640,18.80,0.4,12.0,2021-12-29,29,2021
22644,0.2,-6.8,7.0,0.00,48.5,3.3,7.3,9.140,76.00,-3.9,12.0,2021-12-30,30,2021


In [19]:
X_train , y_train = train[["years", "월", "days","최고기온"]], train.y
X_test, y_test = test[["years", "월", "days","최고기온"]], test.y

In [126]:
X_train , y_train = train.drop(["y", "ds"],axis = 1), train.y
X_test, y_test = test.drop(["y", "ds"],axis = 1), test.y

In [145]:
# XGBoost 모델 초기화 및 학습 (objective="reg:linear"로 설정)
model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
model.fit(X_train, y_train)

# 학습된 모델로 예측
y_pred = model.predict(X_test)

# 예측 결과 평가 (Mean Absolute Error 사용)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 2.840700222278294


In [146]:
# XGBoost 모델 초기화
model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# 튜닝할 하이퍼파라미터 선정
param_grid = {
    'learning_rate': [0.01, 0.05, 0.07, 0.1],
    'max_depth': [3, 4, 6, 8, 10,],
    'n_estimators': [50, 100, 200,300,400,500],
    'subsample': [0.8, 0.9, 0.7],
}

# 그리드 서치를 통한 튜닝
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=3)
grid_result = grid_search.fit(X_train, y_train)

# 최적의 파라미터 출력
print("Best Parameters:", grid_result.best_params_)

# 최적의 모델로 예측
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)

# 예측 결과 평가 (Mean Absolute Error 사용)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error on Test Set: {mae}")

[23:38:44] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:45] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:45] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:45] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:45] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:45] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:20

In [159]:
import pickle
import joblib
# best_model

# 모델 저장
with open('./xgb_model.pkl', 'wb') as model_file:
    pickle.dump(best_model, model_file)
    
# 모델 저장
joblib.dump(model, 'xgb_model.joblib')

['xgb_model.joblib']

In [147]:
params = grid_result.best_params_

model = xgb.XGBRegressor(**params, objective="reg:squarederror", random_state=42)

In [157]:
data_for_summit = data[["years", "월", "days","y"]]
forms = summit_form.copy()
forms.일시 = pd.to_datetime(forms.일시)
forms['days'] = forms.일시.dt.day
forms['월'] = forms.일시.dt.month
forms['years']= forms.일시.dt.year
forms.drop(['일시','평균기온'], axis = 1, inplace = True)

model.fit(data_for_summit.drop(['y'],axis=1), data_for_summit.y)
summit_pred = model.predict(forms)
summit_form.평균기온 = summit_pred
summit_form.to_csv(f"./answer/prophet_mae_{mae}.csv", index = False)

In [133]:
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

In [134]:
## 훈련용

df = train_nofeature.copy()

## xgboost 결과 추가

df["xgb"] = y_pred_train

# Prophet 모델 초기화
model = Prophet(daily_seasonality=True)

model.add_regressor('xgb')

# 모델에 데이터 피팅
model.fit(df)

In [135]:
test_x_xgb = test_x.copy()
test_x_xgb['xgb'] = y_pred_test

In [140]:
default = 2.469308583208345
forecast = model.predict(test_x_xgb)
answer = forecast.yhat
mean_absolute_error(test_y, answer)
mae = mean_absolute_error(test_y, answer)
print(default - mae)

2.1061942857975446


In [139]:
answer = answer[:358]

In [ ]:
## 제출용

df = data[["ds","y"]].copy()


# 모델에 데이터 피팅
model.fit(df)

# 미래 날짜 생성
future = model.make_future_dataframe(periods=358)[-358:]  # 예측할 기간 설정 (365일)

# 예측
forecast = model.predict(future)

summit_form['평균기온']  = model.predict(future).yhat

summit_form.to_csv(f"./answer/prophet_mae_{mae}.csv")

In [142]:
summit_form['평균기온']  = answer

summit_form.to_csv(f"./answer/prophet_mae_{mae}.csv", index = False)

In [160]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV



# 하이퍼파라미터 탐색 범위 정의
param_space = {
    'learning_rate': (0.01, 1.00, 'uniform'),
    'max_depth': (3, 10),
    'n_estimators': (100, 1000),
    'subsample': (0.10, 1.00, 'uniform'),
    'colsample_bytree': (0.10, 1.00, 'uniform'),
    'gamma': (0, 1.0, 'uniform'),
}

# XGBoost 모델 초기화
model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# BayesSearchCV를 사용하여 베이지안 최적화 수행
opt = BayesSearchCV(
    model,
    param_space,
    scoring='neg_mean_absolute_error',
    n_iter=50,  # 반복 횟수
    cv=5,  # 교차 검증 폴드 수
    n_jobs=-1,  # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
)

# 최적의 하이퍼파라미터로 모델 학습
opt.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", opt.best_params_)

# 최적의 모델 평가 (여기서는 교차 검증 점수 출력)
print("Cross-Validation Score:", -opt.best_score_)


Best Parameters: OrderedDict([('colsample_bytree', 0.9062843786292363), ('gamma', 1), ('learning_rate', 0.25906211448112687), ('max_depth', 3), ('n_estimators', 100), ('subsample', 1.0)])
Cross-Validation Score: -0.9014278306654069


In [161]:
model = xgb.XGBRegressor(**opt.best_params_, objective="reg:squarederror", random_state=42)

In [164]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# 예측 결과 평가 (Mean Absolute Error 사용)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error on Test Set: {mae}")

Mean Absolute Error on Test Set: 2.4820028086108703


In [165]:
model.fit(data_for_summit.drop(['y'],axis=1), data_for_summit.y)
summit_pred = model.predict(forms)
summit_form.평균기온 = summit_pred
summit_form.to_csv(f"./answer/prophet_mae_{mae}.csv", index = False)

In [167]:
# 모델 저장
with open(f'./xgb_model_{mae}.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)
    
# 모델 저장
joblib.dump(model, f'xgb_model_{mae}.joblib')

['xgb_model_2.4820028086108703.joblib']

In [ ]:
def summit_fitting(data, model, forms, params):
    
    model = xgb.XGBRegressor(objective="reg:squarederror", **params, random_state=42)
    
    model.fit(data_for_summit.drop(['y'],axis=1), data_for_summit.y)
    
    summit_pred = model.predict(forms)
    
    summit_form.평균기온 = summit_pred
    
    return summit_form.to_csv(f"./answer/prophet_mae_{mae}.csv", index = False)

def summit_train(add):
    ## 리스트형태로 입력
    forms = summit_form.copy()
    forms.일시 = pd.to_datetime(forms.일시)
    forms['days'] = forms.일시.dt.day
    forms['월'] = forms.일시.dt.month
    forms['years']= forms.일시.dt.year
    forms.drop(['일시','평균기온'], axis = 1, inplace = True)
    
    for k,v in add.items():
        
        forms[k] = v
        
    return forms

In [79]:
## 훈련용
# 전체변수를 포함한 데이터셋, 변수리스트, 레이블 값 지정
def prophet_result(train, test, var, y):
    
    var_list = ["ds"]
    var_list.extend(var)
    
    df = train[var_list]

    test_x, test_y = test[var_list],test[y]
    
    df = df.rename(columns = {y : "y"})

    # Prophet 모델 초기화
    model = Prophet(daily_seasonality=True)

    # 모델에 데이터 피팅
    model.fit(df)
    
    forecast = model.predict(test_x)
    answer = forecast.yhat
    print(mean_absolute_error(test_y, answer))
          
    return answer

In [80]:
var =  ["최고기온", "최저기온"]

In [81]:
최고 = prophet_result(train, test, var, "최고기온")
최저 = prophet_result(train, test, var, "최저기온")

14:03:02 - cmdstanpy - INFO - Chain [1] start processing
14:03:06 - cmdstanpy - INFO - Chain [1] done processing


2.9400951176610834


14:03:07 - cmdstanpy - INFO - Chain [1] start processing
14:03:09 - cmdstanpy - INFO - Chain [1] done processing


2.625473120276777


In [32]:
train

,최고기온,y,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온,월,ds,days,years
0,2.2,-5.2,7.4,0.34,68.3,1.7,6.7,7.029,63.10,-1.6,1.0,1960-01-01,1,1960
1,1.2,-5.6,6.8,0.40,87.7,1.3,0.0,2.887,4.19,-1.9,1.0,1960-01-02,2,1960
2,8.7,-2.1,10.8,0.00,81.3,3.0,0.0,5.854,12.87,4.0,1.0,1960-01-03,3,1960
3,10.8,1.2,9.6,0.00,79.7,4.4,2.6,6.279,21.55,7.5,1.0,1960-01-04,4,1960
4,1.3,-8.2,9.5,0.05,44.0,5.1,8.2,13.086,81.51,-4.6,1.0,1960-01-05,5,1960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22641,-3.9,-12.9,9.0,0.00,60.9,1.7,3.8,6.090,39.60,-7.6,12.0,2021-12-27,27,2021
22642,-0.9,-8.5,7.6,0.66,73.8,2.2,1.7,4.320,17.70,-4.1,12.0,2021-12-28,28,2021
22643,5.9,-3.8,9.7,0.20,72.9,2.6,1.8,4.640,18.80,0.4,12.0,2021-12-29,29,2021
22644,0.2,-6.8,7.0,0.00,48.5,3.3,7.3,9.140,76.00,-3.9,12.0,2021-12-30,30,2021


In [28]:
# XGBoost 모델 초기화 및 학습 (objective="reg:linear"로 설정)
model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
model.fit(X_train, y_train)

X_test_ph = X_test.copy()
X_test_ph["최고기온"] = 최고
X_test_ph["최저기온"] = 최저

# 학습된 모델로 예측
y_pred_ph = model.predict(X_test_ph)
y_pred = model.predict(X_test)

# 예측 결과 평가 (Mean Absolute Error 사용)
mae = mean_absolute_error(y_test, y_pred)
mae_ph = mean_absolute_error(y_test, y_pred_ph)
print(f"실제 Mean Absolute Error: {mae}")
print(f"prophet예상 Mean Absolute Error: {mae_ph}")

실제 Mean Absolute Error: 1.0066337131351641
prophet예상 Mean Absolute Error: 27.35552436253796


In [83]:
# XGBoost 모델 초기화 및 학습 (objective="reg:linear"로 설정)
model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
model.fit(X_train, y_train)

X_test_ph = X_test.copy()
X_test_ph["최고기온"] = 최고
X_test_ph["최저기온"] = 최저

# 학습된 모델로 예측
y_pred_ph = model.predict(X_test_ph)
y_pred = model.predict(X_test)

# 예측 결과 평가 (Mean Absolute Error 사용)
mae = mean_absolute_error(y_test, y_pred)
mae_ph = mean_absolute_error(y_test, y_pred_ph)
print(f"실제 Mean Absolute Error: {mae}")
print(f"prophet예상 Mean Absolute Error: {mae_ph}")

실제 Mean Absolute Error: 0.41303981013494
prophet예상 Mean Absolute Error: 28.86163728949142


In [24]:
X_test

,years,월,days,최고기온
22646,2022,1.0,1,2.3
22647,2022,1.0,2,3.0
22648,2022,1.0,3,2.5
22649,2022,1.0,4,1.0
22650,2022,1.0,5,1.9
...,...,...,...,...
23006,2022,12.0,27,3.3
23007,2022,12.0,28,0.1
23008,2022,12.0,29,2.1
23009,2022,12.0,30,2.3
